In [1]:
import numpy as np
import pandas as pd

In [2]:
df_rating = pd.read_pickle('players_rating_clean.zip')
df_players = pd.read_csv('titled_players.csv', index_col=0)

In [3]:
df_merged = df_rating.merge(df_players[['id', 'nom', 'titre', 'pays', 'naissance', 'sexe', 'actif']], how='left', on='id')

In [4]:
df_merged.sample(5)

,id,date,classique_elo,classique_parties,rapide_elo,rapide_parties,blitz_elo,blitz_parties,nom,titre,pays,naissance,sexe,actif
1136185,11600551,2009-11-01,2334.0,0.0,NaN,NaN,NaN,NaN,"Jankovskis, Guntis",FM,LAT,1971.0,M,1
741871,1600524,2019-03-01,2157.0,0.0,NaN,NaN,2218.0,0.0,"Kranzl, Peter",FM,AUT,1964.0,M,1
1618645,108987,2018-02-01,1937.0,0.0,2016.0,0.0,2089.0,0.0,"Maggiolo, Elisa",WIM,ARG,1980.0,F,1
2731389,25072846,2014-11-01,1846.0,9.0,1780.0,0.0,1773.0,9.0,Aronyak Ghosh,IM,IND,2003.0,M,1
2008377,13603353,2015-01-01,2300.0,0.0,NaN,NaN,NaN,NaN,"Izoria, Giorgi",FM,GEO,1987.0,M,0


In [5]:
df_model = df_merged[df_merged['date'] >= '2013-01-01'][df_merged[df_merged['date'] >= '2013-01-01']['classique_elo'].notna()]

In [6]:
df_model.shape[0] / df_rating.shape[0]

0.6952351499332294

In [7]:
df_model.head()

,id,date,classique_elo,classique_parties,rapide_elo,rapide_parties,blitz_elo,blitz_parties,nom,titre,pays,naissance,sexe,actif
0,4611870,2021-09-01,2290.0,0.0,NaN,NaN,NaN,NaN,"Keller-Hermann, Edith",WGM,GER,1921.0,F,0
1,4611870,2021-08-01,2290.0,0.0,NaN,NaN,NaN,NaN,"Keller-Hermann, Edith",WGM,GER,1921.0,F,0
2,4611870,2021-07-01,2290.0,0.0,NaN,NaN,NaN,NaN,"Keller-Hermann, Edith",WGM,GER,1921.0,F,0
3,4611870,2021-06-01,2290.0,0.0,NaN,NaN,NaN,NaN,"Keller-Hermann, Edith",WGM,GER,1921.0,F,0
4,4611870,2021-05-01,2290.0,0.0,NaN,NaN,NaN,NaN,"Keller-Hermann, Edith",WGM,GER,1921.0,F,0


Essayons de prédire le sexe d'un joueur suivant son classement, âge 

In [8]:
df_first_attempt = df_model[['id', 'classique_elo', 'classique_parties', 'naissance', 'sexe']]

In [9]:
df_first_attempt['sexe'].value_counts(normalize=True)

M    0.804803
F    0.195197
Name: sexe, dtype: float64

In [10]:
u = df_first_attempt.groupby('id').agg({'classique_elo':np.mean, 'classique_parties':np.mean, 'naissance':np.mean,'sexe':np.max})

In [11]:
z = df_first_attempt.groupby('id').std(ddof=0)[['classique_elo', 'classique_parties']]

In [12]:
z.columns = ['classique_elo_std', 'classique_parties_std']

In [13]:
a = u.merge(z, how='left', on='id').merge(df_players[['id','pays','actif']], how='left', on='id')

In [14]:
a = a[['classique_elo','classique_elo_std', 'classique_parties', 'classique_parties_std', 'naissance', 'sexe', 'pays','actif']]

In [15]:
a = pd.get_dummies(a, columns=['pays'])
a.head()

,classique_elo,classique_elo_std,classique_parties,classique_parties_std,naissance,sexe,actif,pays_AFG,pays_AHO,pays_ALB,...,pays_UKR,pays_URU,pays_USA,pays_UZB,pays_VEN,pays_VIE,pays_WLS,pays_YEM,pays_ZAM,pays_ZIM
0,2437.523810,21.231197,5.457143,7.008877,1957.0,M,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2422.000000,0.000000,0.000000,0.000000,1947.0,M,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2435.361905,29.320925,2.104762,4.040094,1968.0,M,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2425.000000,0.000000,0.000000,0.000000,1958.0,M,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2340.000000,0.000000,0.000000,0.000000,1958.0,M,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
df_train, df_test = train_test_split(a, test_size=0.2, random_state=10)

In [23]:
X = a.copy()
y = X.pop('sexe')
scores = cross_val_score(RandomForestClassifier(), X, y, cv=5, scoring='f1_micro')

In [35]:
y

0        M
1        M
2        M
3        M
4        M
        ..
19428    M
19429    F
19430    M
19431    M
19432    M
Name: sexe, Length: 19433, dtype: object

In [24]:
scores

array([0.83483406, 0.64214047, 0.60535117, 0.6618631 , 0.73237262])

In [59]:
temp = df_train.copy()
temp.pop('sexe')
cols = temp.columns
cols = ['classique_elo' ,'classique_elo_std' , 'classique_parties','classique_parties_std','naissance','actif']
X_train = df_train[cols]
y_train = df_train['sexe']
X_test = df_test[cols]
y_test = df_test['sexe']


In [32]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [33]:
f1_score(y_test, predictions, average='micro')

0.882943143812709

RandomForestClassifier pour un cross validation en 5 splits

In [43]:
skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
score = []
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    score.append(f1_score(y_test, predictions, average='micro'))


In [52]:
print(f"5 split score : {score},\n score moyen:{np.array(score).mean()}")

5 split score : [0.8790841265757655, 0.8862876254180602, 0.881914072549524, 0.8872876994338652, 0.8860010293360784],
 score moyen:0.8841149106626587


Linear 

In [69]:
from sklearn.linear_model import LogisticRegression

skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
score2 = []
# X_temp = X[cols]
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model = LogisticRegression(max_iter=500)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    score2.append(f1_score(y_test, predictions, average='micro'))


In [68]:
print(f"5 split score : {score2},\n score moyen:{np.array(score2).mean()}")

5 split score : [0.8572163622330846, 0.8600463082068434, 0.8592745047594545, 0.8561502830674214, 0.8589809572825527],
 score moyen:0.8583336831098712
